In [1]:
import imageio
from PIL import Image
import numpy as np

# ---------------- SETTINGS ----------------
gif_paths = [
    "induction_equation_1_year_diff200.gif",
    "induction_equation_1_year_diff400.gif",
    "induction_equation_1_year_diff600.gif",
    "induction_equation_1_year_diff800.gif",
]

output_mp4 = "combined_column_diff_const.mp4"
target_width = 600   # resize width (px)
fps = 2             # output video FPS
# ------------------------------------------

# Load GIFs as frame lists
gif_frames = []
gif_durations = []

for path in gif_paths:
    reader = imageio.get_reader(path)
    frames = []
    durations = []
    for frame in reader:
        frames.append(Image.fromarray(frame))
        durations.append(reader.get_meta_data().get("duration", 50) / 1000)
    gif_frames.append(frames)
    gif_durations.append(sum(durations))

# Compute max duration to sync loops
max_dur = max(gif_durations)

# Compute total number of output frames
num_frames = int(max_dur * fps)

# Prepare resized GIF frame sequences
resized_sequences = []

for frames in gif_frames:
    # resize all frames to same width
    resized = []
    for frame in frames:
        w, h = frame.size
        scale = target_width / w
        new_h = int(h * scale)
        resized.append(frame.resize((target_width, new_h), Image.LANCZOS))
    resized_sequences.append(resized)

# Build the video
writer = imageio.get_writer(output_mp4, fps=fps)

for idx in range(num_frames):
    column_frames = []
    for seq in resized_sequences:
        # Loop each gif
        frame = seq[idx % len(seq)]
        column_frames.append(np.array(frame))

    # stack vertically
    combined = np.vstack(column_frames)
    writer.append_data(combined)

writer.close()
print("Saved:", output_mp4)


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 1800) to (608, 1808) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[rawvideo @ 0x218a7d00] Stream #0: not enough frames to estimate rate; consider increasing probesize


Saved: combined_column_diff_const.mp4
